In [0]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import glob
from google.colab import drive
import pickle
import librosa
from itertools import combinations 
import keras

In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
f = open('/content/drive/My Drive/assignment4/hw4_trs.pkl','rb') 
train = pickle.load(f)
f = open('/content/drive/My Drive/assignment4/hw4_tes.pkl','rb') 
test = pickle.load(f)

Defining Parameters

In [0]:
classes = 512
hiddenunits = 512
batchsize = 90
epochs = 100
keep_prob = 0.9
learning_rate = 0.0001
dropout = 0.1

Creating the dataset from read data

In [0]:
train1 = [np.abs(librosa.stft(s, n_fft=1024, hop_length=512)).T for s in train]
test1 = [np.abs(librosa.stft(s, n_fft=1024, hop_length=512)).T for s in test]

In [0]:
def datagen(data):
  sp_no = int(len(data)/10)
  out1 = []
  out2 = []
  y  = []  
  for i in range(1,sp_no+1):
    curr = data[i*10 - 10:i*10]
    rest = np.delete(data,range(i*10 - 10,i*10),axis=0)
    p = list(combinations(range(10), 2))
    out1 = out1 +  [curr[i[0]] for i in p]
    out2 = out2 + [curr[i[1]] for i in p]
    y  = y + [1]*len(p)
    l = list(zip(np.random.randint(10,size=45),np.random.randint(len(data)-10,size=45)))
    out1 = out1 +  [curr[i[0]] for i in l]
    out2 = out2 + [rest[i[1]] for i in l]
    y  = y + [0]*len(l)
  s = np.arange(np.array(out1).shape[0])
  np.random.shuffle(s)
  return np.array(out1)[s],np.array(out2)[s],np.array(y)[s]

Dataset generation

In [0]:
train_1,train_2,train_y = datagen(train1)
test_1,test_2,test_y = datagen(test1)

In [0]:
trainl=[]
for i in train_1:
  trainl.append(i.shape[0])

In [0]:
testl=[]
for i in test_1:
  testl.append(i.shape[0])

In [0]:
iterations = int(len(train_1)/batchsize)

Batch generation

In [0]:
def batchgen(t1,t2,seq_len,st,batch_size,y):
     return t1[st:st + batch_size],t2[st:st + batch_size],seq_len[st:st + batch_size],y[st:st + batch_size]

Defining LSTM CELL

In [0]:
def lstmcell():
    lstm =tf.nn.rnn_cell.DropoutWrapper(tf.nn.rnn_cell.GRUCell(hiddenunits,kernel_initializer =tf.keras.initializers.glorot_normal()),output_keep_prob = 1 - dropout)
    return lstm

Setting up Graph

In [0]:
tf.reset_default_graph()
X1 = tf.placeholder(tf.float32,[None,None,513])
X2 = tf.placeholder(tf.float32,[None,None,513])
Y = tf.placeholder(tf.float32,[None,1])
traindec=tf.placeholder(tf.bool)
seq_len = tf.placeholder(tf.int32,None)

Defining model

In [0]:
def siamese_model(inp,traindec):
    with tf.name_scope("Model"):
        with tf.variable_scope("rnn",reuse = tf.AUTO_REUSE ) as scope:
              cell = tf.contrib.rnn.MultiRNNCell([lstmcell() for _ in range(3)])
              network, _  = tf.nn.dynamic_rnn(cell, inp, dtype=tf.float32,sequence_length=seq_len)
        with tf.variable_scope("dense",reuse = tf.AUTO_REUSE ) as scope: 
              network = tf.layers.batch_normalization(inputs=network,axis=-1,momentum=0.8,epsilon=0.0001,reuse= False,training = traindec)
              network = tf.layers.dense(network, classes, kernel_initializer=tf.keras.initializers.glorot_normal(),activation=tf.nn.tanh)
              network = tf.layers.flatten(network)
        return network

In [0]:
op1 =  tf.nn.l2_normalize(siamese_model(X1,traindec),0)
op2 =  tf.nn.l2_normalize(siamese_model(X2,traindec),0)
final_layer =tf.sigmoid(tf.reduce_sum(tf.multiply( op1, op2 ),1, keep_dims=True))

In [0]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=final_layer))
train_optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(final_layer),Y),tf.float32))

Training and Testing the model

In [47]:
with tf.Session()  as sess:
  cost = np.zeros(epochs)
  tf.global_variables_initializer().run() 
  tf.local_variables_initializer().run()
  update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
  for i in range(epochs):
    for itr in range(iterations):
      batch_1,batch_2,seq_len_batch,batch_y = batchgen(train_1,train_2,trainl,itr*batchsize,batchsize,train_y)
      batch_y = batch_y.reshape([batchsize,1])
      _,_,_,tr_loss = sess.run([train_optimizer,accuracy,update_ops,loss], feed_dict={X1:batch_1,X2:batch_2,Y:batch_y,seq_len:seq_len_batch,traindec:True})
    cost[i],acc = sess.run([loss,accuracy],feed_dict={X1:test_1,X2:test_2,Y:test_y.reshape([len(test_y),1]),seq_len:testl,traindec:False})
    if i%10 == 0:      
      print("Loss for epoch",str(i),"=", str(cost[i]), "With Accuracy =", str(acc)) 

Loss for epoch 0 = 0.799278974533081 With Accuracy = 0.5
Loss for epoch 10 = 0.7463143467903137 With Accuracy = 0.52444446
Loss for epoch 20 = 0.7065346837043762 With Accuracy = 0.6338889
Loss for epoch 30 = 0.7214862704277039 With Accuracy = 0.60444444
Loss for epoch 40 = 0.6944419741630554 With Accuracy = 0.675
Loss for epoch 50 = 0.7426771521568298 With Accuracy = 0.5644444
Loss for epoch 60 = 0.6809706091880798 With Accuracy = 0.69222224
Loss for epoch 70 = 0.6760926842689514 With Accuracy = 0.69777775
Loss for epoch 80 = 0.6995741128921509 With Accuracy = 0.63666666
Loss for epoch 90 = 0.668730616569519 With Accuracy = 0.7183333


Accuracy the test set for 100 iterations is 72%

